In [ ]:
import ipywidgets.widgets as widgets
from time import sleep
import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')
from ballsbot.camera import get_images_and_cameras
from ballsbot.controller import link_controller
from ballsbot.utils import keep_rps, run_as_thread
from ballsbot.augmented_lidar import get_augmented_lidar
from ballsbot.ros_messages import get_ros_messages
from ballsbot.manipulator import Manipulator
from ballsbot.joystick import JoystickWrapper
from ballsbot.drawing import ManipulatorPoseDrawing
from ballsbot.lidar_drawing import LidarDrawing

In [ ]:
get_ros_messages().start(sync=False)

In [ ]:
controller = widgets.Controller(index=0)  # http://html5gamepad.com
display(controller)

images = []
for image, camera in get_images_and_cameras():
    images.append(image)
hbox = widgets.HBox(images)
display(hbox)

lidar_image = widgets.Image(format='jpeg', width=600, height=600)

painter = ManipulatorPoseDrawing(widgets)
display(widgets.HBox([
    lidar_image,
    widgets.VBox([painter.get_images(), widgets.Image(format='jpeg', width=400, height=200)])
]))

In [ ]:
joystick_wrapper = JoystickWrapper(controller)
manipulator = Manipulator(joystick_wrapper)

link_controller(joystick_wrapper)

lidar = get_augmented_lidar()
lidar.start()

# lidar = Lidar()
# lidar.calibrate()
# print(lidar.calibration)

lidar_drawer = LidarDrawing(lidar)

In [ ]:
def lidar_drawer_run():
    lidar_drawer.show_lidar_cloud(lidar_image, cached=False)

run_as_thread(lidar_drawer_run)

In [ ]:
def update_manipulator_images():
    ts = None
    while True:
        ts = keep_rps(ts, fps=1)
        pose = manipulator.get_manipulator_pose()
        painter.update_images(pose, override_crop_half_size=550.)

run_as_thread(update_manipulator_images)

In [ ]:
# from ballsbot.utils import join_all_theads
# join_all_theads()